In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated_umap.csv"
).resolve()
# shiny output path
shiny_output_path = pathlib.Path(
    "../temporal_shiny_app/CLS_features_annotated_umap.csv"
).resolve()

output_path.parent.mkdir(parents=True, exist_ok=True)
shiny_output_path.parent.mkdir(parents=True, exist_ok=True)

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_original_index,Metadata_compound,Metadata_dose,Metadata_control,...,channel_DNA_cls_feature_90,channel_DNA_cls_feature_91,channel_DNA_cls_feature_92,channel_DNA_cls_feature_93,channel_DNA_cls_feature_94,channel_DNA_cls_feature_95,channel_DNA_cls_feature_96,channel_DNA_cls_feature_97,channel_DNA_cls_feature_98,channel_DNA_cls_feature_99
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102.0,145086,Staurosporine,0.0,negative,...,0.035033,-0.000629,-0.034413,0.056733,-0.078023,0.033735,0.006015,0.016154,0.036660,-0.014219
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106.0,145087,Staurosporine,0.0,negative,...,-0.005406,-0.033099,-0.042950,0.012875,-0.074455,0.030203,0.032813,-0.017182,0.052587,-0.005798
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,122.0,145088,Staurosporine,0.0,negative,...,0.015690,0.027015,0.001716,0.039788,-0.119122,0.010958,0.007134,0.048436,0.046638,0.018560
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,128.0,145090,Staurosporine,0.0,negative,...,-0.009345,-0.016075,-0.035206,0.037123,-0.102122,-0.030821,0.027550,0.001286,0.091023,-0.000872
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13.0,145073,Staurosporine,0.0,negative,...,0.026046,-0.004125,0.002168,0.033040,-0.054002,0.024171,-0.018173,0.041654,0.045350,-0.033588


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (140235, 10)
feature_df shape: (140235, 1536)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniforge3/envs/scDINO_env/lib/python3.8/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_original_index,Metadata_compound,Metadata_dose,Metadata_control,UMAP1,UMAP2
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102.0,145086,Staurosporine,0.0,negative,0.798541,2.473827
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106.0,145087,Staurosporine,0.0,negative,0.891429,1.486087
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,122.0,145088,Staurosporine,0.0,negative,0.905020,2.480158
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,128.0,145090,Staurosporine,0.0,negative,-3.307871,-0.311870
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13.0,145073,Staurosporine,0.0,negative,1.629274,1.221360


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)

# save the UMAP embeddings to shiny app
umap_embedding_df.to_csv(shiny_output_path)